For each set of NEs predicted by each package (i.e. Stanza and Spacy), how many of them can be found mentioned in the KG Graph you ollected for the person?7 

return for each package (i.e. Stanza and Spacy), the ratio of
predicted NEs that you can be confidently said to be in the KG
graph for the person.

7 note that you should take into consid-
eration the NE span in the text may not
be an exact match with KG entity label.
Here you can use regular expressions
to match the NEs found in the text
with the RDF entities present in the
associated texts.

In [31]:
import os 
import pandas as pd 
import numpy as np
import json
import re

In [36]:
entities_stanza

,text,n_word,start_pos,end_pos,label,subject,source_index,category,range_span
0,Wilhelm Conrad Röntgen,3,0,22,PERSON,Wilhelm Röntgen,0,Physics,[ 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 ...
1,German,1,26,32,NORP,Wilhelm Röntgen,0,Physics,[26 27 28 29 30 31 32]
2,27 March 1845,3,70,83,DATE,Wilhelm Röntgen,0,Physics,[70 71 72 73 74 75 76 77 78 79 80 81 82 83]
3,10 February 1923,3,86,102,DATE,Wilhelm Röntgen,0,Physics,[ 86 87 88 89 90 91 92 93 94 95 96 ...
4,German,1,110,116,NORP,Wilhelm Röntgen,0,Physics,[110 111 112 113 114 115 116]
...,...,...,...,...,...,...,...,...,...
59866,PDF,1,5268,5271,ORG,Jerome Karle,199,Chemistry,[5268 5269 5270 5271]
59867,Philadelphia,1,5274,5286,GPE,Jerome Karle,199,Chemistry,[5274 5275 5276 5277 5278 5279 5280 5281 5282 ...
59868,Science History Institute,3,5292,5317,ORG,Jerome Karle,199,Chemistry,[5292 5293 5294 5295 5296 5297 5298 5299 5300 ...
59869,Jerome Karle,2,5319,5331,PERSON,Jerome Karle,199,Chemistry,[5319 5320 5321 5322 5323 5324 5325 5326 5327 ...


In [7]:
par_dir = os.path.dirname(os.getcwd())
data_dir = os.path.join(par_dir, '2.2 NER - analysis by entity type')

stanza_dir = os.path.join(data_dir, "df_entities_spacy_processed.csv")
spacy_dir = os.path.join(data_dir, "df_entities_stanza_processed.csv")

entities_stanza = pd.read_csv(stanza_dir)
entities_spacy = pd.read_csv(spacy_dir)

In [8]:
list_people = list(set(list(np.unique(entities_spacy["subject"])) + list(np.unique(entities_stanza["subject"]))))
list_entities = list(set(list(np.unique(entities_spacy["label"])) + list(np.unique(entities_stanza["label"]))))

In [37]:

def make_regex(entity):
    
    entity_diff_start_end = "[0-9]{0,2} " + entity +  " [0-9]{0,2} " # to account for dates where there may only be a year in the graph but a full date in the entity
    entity_underscore = "_?[a-z]{0,9}_?".join(entity_diff_start_end.split(" ")) # adding optional underscores and words from length 0-9
    
    return entity_underscore

In [49]:
for person in list_people[0:3]:
    
    # filtering each df by the subject of the bio
    filter_df_spacy = entities_spacy[entities_spacy["subject"] == person]
    filter_df_stanza = entities_stanza[entities_stanza["subject"] == person]
    
    # getting the category to know which Data Directory subfolder we have to access
    category = filter_df_spacy["category"].iloc[0]
    
    # print(category)
    # getting the entities found in the bios per package
    set_entities_spacy = set(filter_df_spacy["text"].to_list())
    set_entities_stanza = set(filter_df_stanza["text"].to_list())
    
    # total amount of unique entities
    total_entities_spacy = len(set_entities_spacy)
    total_entities_stanza = len(set_entities_stanza)
    
    found_entities_spacy = 0
    found_entities_stanza = 0   
    
    # loading the json graphs to query them
    par_dir = os.path.dirname(os.path.dirname(os.getcwd()))
    name_file_graph = person.replace(" ", "__") + ".json"
    data_dir = os.path.join(par_dir, "Data Directory")
    
    if category == "Chemistry":
        graph_dir = os.path.join(data_dir, "chemistry_nobel_laureate")
    
    else:
        graph_dir = os.path.join(data_dir, "physics_nobel_laureate")
        
    person_graph_path = os.path.join(graph_dir, name_file_graph)
    # print(person_graph_path)
    
    with open(person_graph_path, "r") as f:
        personal_graph = json.load(f)
    # if f:
    #     print("Graph load successfully!")
    
    list_subjects = [dict_graph["subject"]["value"].strip("http://dbpedia.org/resource/") for dict_graph in personal_graph]
    list_objects = [dict_graph["object"]["value"].strip("http://dbpedia.org/resource/") for dict_graph in personal_graph]
    
    list_entities_graph = list(set(list_subjects + list_objects))
    
    # print(list_entities_graph)
    
    regex_entities_stanza = [make_regex(ent) for ent in set_entities_stanza]
    regex_entities_spacy = [make_regex(ent) for ent in set_entities_spacy]
    
    matches_spacy = 0
    
    print(regex_entities_stanza)
    for ent in list_entities_graph:
        
        match_search = [(re.search(regex, ent, flags=re.IGNORECASE), regex) for regex in regex_entities_spacy]
        match = [(search[0].group(0), search[1]) for search in match_search if search[0] != None]
        print(ent)
        print(match)
        if match:
            matches = matches_spacy + 1
        
        print(matches)
    prop_matches = round((matches_spacy/total_entities_spacy), 2)
    
    

['[0-9]{0,2}_?[a-z]{0,9}_?John_?[a-z]{0,9}_?Cowdery_?[a-z]{0,9}_?Kendrew_?[a-z]{0,9}_?[0-9]{0,2}_?[a-z]{0,9}_?', '[0-9]{0,2}_?[a-z]{0,9}_?1936_?[a-z]{0,9}_?[0-9]{0,2}_?[a-z]{0,9}_?', '[0-9]{0,2}_?[a-z]{0,9}_?1947_?[a-z]{0,9}_?[0-9]{0,2}_?[a-z]{0,9}_?', '[0-9]{0,2}_?[a-z]{0,9}_?the_?[a-z]{0,9}_?Royal_?[a-z]{0,9}_?Institution_?[a-z]{0,9}_?[0-9]{0,2}_?[a-z]{0,9}_?', '[0-9]{0,2}_?[a-z]{0,9}_?Cambridge_?[a-z]{0,9}_?[0-9]{0,2}_?[a-z]{0,9}_?', '[0-9]{0,2}_?[a-z]{0,9}_?the_?[a-z]{0,9}_?European_?[a-z]{0,9}_?Molecular_?[a-z]{0,9}_?Biology_?[a-z]{0,9}_?Organization_?[a-z]{0,9}_?[0-9]{0,2}_?[a-z]{0,9}_?', '[0-9]{0,2}_?[a-z]{0,9}_?John_?[a-z]{0,9}_?Finch_?[a-z]{0,9}_?[0-9]{0,2}_?[a-z]{0,9}_?', '[0-9]{0,2}_?[a-z]{0,9}_?the_?[a-z]{0,9}_?International_?[a-z]{0,9}_?Academy_?[a-z]{0,9}_?of_?[a-z]{0,9}_?Science,_?[a-z]{0,9}_?Munich_?[a-z]{0,9}_?[0-9]{0,2}_?[a-z]{0,9}_?', '[0-9]{0,2}_?[a-z]{0,9}_?Nobel_?[a-z]{0,9}_?Prize_?[a-z]{0,9}_?[0-9]{0,2}_?[a-z]{0,9}_?', '[0-9]{0,2}_?[a-z]{0,9}_?Elizabeth_?[a-z]{0,

error: multiple repeat at position 141